# Neptune Analytics Instance Management Full Lifecycle Demo

This notebook demonstrates how to:
1. Run an algorithm (pagerank) on Neptune Analytics, including instance creation and destroy.
2. Run an algorithm (pagerank) on Neptune Analytics against a CSV file and an existing graph instance.
3. Run a mutating algorithm (pagerank) on Neptune Analytics against a CSV file and export the results to S3 as a CSV.

This demo uses the backend configuration to manage the entire workflow.

## Setup

Import the necessary libraries and set up logging.

In [ ]:
import logging
import sys

import networkx as nx

In [ ]:
# Configure logging to see detailed information about the instance creation process
logging.basicConfig(
    level=logging.INFO,
    format='%(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
    stream=sys.stdout  # Explicitly set output to stdout
)
for logger_name in ['nx_neptune.utils.decorators']:
    logging.getLogger(logger_name).setLevel(logging.DEBUG)

## Example 1

In this example, we will run networkx pagerank algorithm in Neptune Analytics without an existing instance being defined.  The workflow is defined in the configuration, and the configuration will create a new neptune-graph instance, sync data, and run pagerank.

We will re-use the graph instance in the following examples.

In [ ]:
nx.config.backends.neptune.create_new_instance = False
# Or with custom configuration, detail usage can be found under:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/neptune-graph/client/create_graph.html
# nx.config.backends.neptune.create_new_instance = {
#     "provisionedMemory": 32,
#     "tags": {"additional_tag": "test_with_additional_tag"},
# }
nx.config.backends.neptune.graph_id = "<GRAPH_ID>"
nx.config.backends.neptune.destroy_instance = False

g = nx.DiGraph()
g.add_node("Alice")
g.add_node("Bob")
g.add_node("Carl")
g.add_edge("Alice", "Bob")
g.add_edge("Alice", "Carl")

BACKEND = "neptune"

r_neptune = nx.pagerank(g, backend="neptune")
print("PageRank results using Neptune Analytics:")
sorted_results = sorted(r_neptune.items(), key=lambda x: (x[1], x[0]), reverse=True)
for key, value in sorted_results[:10]:
    print(f"{key}: {value:.6f}")

## Example 2

In this example, we will run networkx pagerank algorithm in Neptune Analytics against an already existing graph instance.  Before running the algorithm, we will import results from S3.

In [ ]:
nx.config.backends.neptune.create_new_instance = False
# define this value if the previous step was not run; otherwise it will be defined automatically
# nx.config.backends.neptune.graph_id = "<GRAPH_ID>"
nx.config.backends.neptune.skip_graph_reset = True
nx.config.backends.neptune.import_s3_bucket = "<your-s3-bucket>/cit-Patents"
nx.config.backends.neptune.s3_iam_role = "<your-role>"
nx.config.backends.neptune.destroy_instance = False

print(f"Using instance graph: {nx.config.backends.neptune.graph_id}")

g = nx.DiGraph()

BACKEND = "neptune"

r_neptune = nx.pagerank(g, backend="neptune")
print("PageRank results using Neptune Analytics:")
sorted_results = sorted(r_neptune.items(), key=lambda x: (x[1], x[0]), reverse=True)
for key, value in sorted_results[:10]:
    print(f"{key}: {value:.6f}")

## Example 3

In this example, we will run networkx pagerank algorithm in Neptune Analytics using an existing instance.  We will run clean up operations in the workflow.  Once the algorithm is complete, we will export the graph contents to S3 as a Gremlin CSV, and destroy the instance.

In [ ]:
nx.config.backends.neptune.create_new_instance = False
# define this value if the previous step was not run; otherwise it will be defined automatically
# nx.config.backends.neptune.graph_id = "<GRAPH_ID>"
nx.config.backends.neptune.import_s3_bucket = None
nx.config.backends.neptune.export_s3_bucket = "<your-s3-bucket>/export"
nx.config.backends.neptune.s3_iam_role = "<your-role>"
nx.config.backends.neptune.destroy_instance = True

print(f"Using instance graph: {nx.config.backends.neptune.graph_id}")

g = nx.DiGraph()

BACKEND = "neptune"

r_neptune = nx.pagerank(g, backend="neptune")
print("PageRank results using Neptune Analytics:")
sorted_results = sorted(r_neptune.items(), key=lambda x: (x[1], x[0]), reverse=True)
for key, value in sorted_results[:10]:
    print(f"{key}: {value:.6f}")

## Conclusion

This notebook demonstrated the complete lifecycle of a Neptune Analytics instance:

1. **Creation**: We created a new Neptune Analytics instance on demand.
2. **Re-use Graph** and **Import**: We imported the demo Citations dataset from S3 into the newly created instance.
3. **Export** and **Deletion**: We export the demo Citations dataset to S3 and deleted the on demand instance after the computation is completed.